In [1]:
!python --version

Python 3.8.5


In [2]:
import pprint
import sys
pprint.pprint(sys.path)

['/Users/kannomasato/Desktop/jupyter',
 '',
 '/Users/kannomasato/opt/anaconda3/envs/my_notebook_env/lib/python3.8/site-packages',
 '/Users/kannomasato/Desktop/jupyter',
 '/Users/kannomasato/opt/anaconda3/lib/python38.zip',
 '/Users/kannomasato/opt/anaconda3/lib/python3.8',
 '/Users/kannomasato/opt/anaconda3/lib/python3.8/lib-dynload',
 '/Users/kannomasato/opt/anaconda3/lib/python3.8/site-packages',
 '/Users/kannomasato/opt/anaconda3/lib/python3.8/site-packages/aeosa',
 '/Users/kannomasato/opt/anaconda3/envs/my_notebook_env/lib/python3.8/site-packages/IPython/extensions',
 '/Users/kannomasato/.ipython']


In [3]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing, metrics, model_selection
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer 
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, train_test_split
from datetime import datetime, date, timezone, timedelta
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc

In [5]:
PATH = '/Users/kannomasato/Desktop/jupyter/pp_gas_emission/'

## ファイル読み込み
data = pd.read_csv(PATH+'gt_all.csv')

In [6]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.4 ,shuffle=False)
train_data, validation_data = train_test_split(train_data, train_size=0.66 ,shuffle=False)

no_label = "CO"
train_data = train_data.drop(no_label,axis=1)
test_data = test_data.drop(no_label,axis=1)
validation_data = validation_data.drop(no_label,axis=1)

label =  "NOX"
 
test_labels = test_data.pop(label).values

In [7]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             primary_metric='r2_score',
                             experiment_timeout_minutes=60,
                             training_data=train_data,
                             label_column_name=label,
                             validation_data = validation_data,
                             debug_log='automated_ml_errors.log')

In [8]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "pp-NOX-emission-experiment")
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_7f359b65-1150-42aa-b7af-0e3bda3a4eab

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and 

WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='japaneast.experiments.azureml.net', port=443): Read timed out. (read timeout=100)")': /history/v1.0/subscriptions/622efba4-7967-4d6d-b3d5-1f03484df247/resourceGroups/pf_automl_portal_dev/providers/Microsoft.MachineLearningServices/workspaces/automl-portal-dev/experiments/pp-NOX-emission-experiment/runs:query


****************************************************************************************************
Current status: BestRunExplainModel. Best run model explanations started
Current status: ModelExplanationDataSetSetup. Model explanations data setup completed
Current status: PickSurrogateModel. Choosing LightGBM as the surrogate model for explanations
Current status: EngineeredFeatureExplanations. Computation of engineered features started
Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
****************************************************************************************************


In [21]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…